<a href="https://colab.research.google.com/github/BrunoMog/PROJETOS-IF687/blob/main/mlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Início da implementação


In [1]:
#importanto biblioteca
import numpy as np

In [23]:
#setando parâmetros da rede mlp

input_size = 2
hidden_size = 5
output_size = 7

#cada camada intermediária pode ter o mesmo número de unidades computacionais para fins de simplificação ou apenas terem valores diferentes, nessa implementação vamos adotar a segunda abordagem
#para cada camada intermediária - hidden layer, terá um valor relacionado a quantidade de unidades computacionais

hidden_layer_size = [5, 5, 5, 5, 5]

# serão implementadas três funções de ativação para as camadas intermediárias a relu, sigmoid e linear pela qual será indicada, e para a camdada de saída conforme o indicado sigmoid ou linear
# mas a relu também é uma opção
# a primeira função será para camadas intermediárias e a segunda para a camada de saída

activation_functions = ['relu', 'relu']

# o número máximo de épocas que o modelo será treinado
# o objetivo é que não precise chegar ao número máximo de épocas, quando o modelo apresentar overfitting o treinamento já acabe

MAX_EPOCH = 1

# taxa de aprendizado que será utilizada

learning_rate = 0.03

# tamanho dos mini-batchs que será utilizado dos dados de treinamento e teste

batch_size_train = 100

batch_size_test = 20

In [24]:
# setando o gerador de números aleatórios para setar os pesos iniciais

weigths_seed = 115
rng = np.random.default_rng(weigths_seed)

# como a rede mlp é totalmente conectada, o número de pesos gerados entre camadas segirá sempre o padrão layer[i]*layer[i+1]

weights = []

for i in range(hidden_size+1):

  if i == 0:

    weights.append(rng.random(size = (input_size, hidden_layer_size[i], 1)))

  elif i < hidden_size:

    weights.append(rng.random((hidden_layer_size[i-1], hidden_layer_size[i], 1)))

  else:

    weights.append(rng.random((hidden_layer_size[i-1], output_size, 1)))



In [31]:
# primeiro index é para referir a qual camada, sempre haverá hidden_layers+1 camada de pesos
# o segundo index para referenciar qual unidade computacional da camada
# o terceiro index para referenciar o peso a qual unidade computacional da próxima camada
# o quarto index para obter o valor

print(len(weights[-1][-1]))

7


### Funções para o treinamento da rede MLP

#### fase foward e backward


In [5]:
# funções de ativação

# função relu normal
def function_Relu(x):

  return max(0, x)

# função sigmoid normal
def function_Sigmoid(x):

  return 1/(1+np.exp(-x))

# função linear entre 0 e 1
def function_Linear(x):

  return max(0, min(1, x))

In [ ]:
# derivada das funções de ativação

def derivative_Relu(x):

  if x > 0:

    return 1

  else:

    return 0

# no caso da derivada da sigmoid, como o input vai ser o erro gerado pela aquiela unidade computacional
# a derivada da sigmoide também pode se expressa por sigmoid(x)*(1-sigmoid(x)), e como sigmoid(x) é o input o output pode ser expresõ por x*(1-x)
def derivative_Sigmoid(x):

  return x*(1-x)

def derivative_Linear(x):

  if x > 0 and x < 1:

    return 1

  else:

    return 0

In [6]:
# funções de custo

def mean_square_loss(labels, predictions):

  loss = 0

  for l, p in zip(labels, predictions):
      loss = loss + (l - p) ** 2

  loss = loss / len(labels)

  return loss

In [7]:
# processo de foward da rede

def foward(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, weights, data):

  resultado_final = []

  #pra cada dado recebido como entrada, gera o valor da fase foward e armazena em resultado_final

  for m in range(len(data)):

    # o resultado de cada valor multiplicado pelo seu peso será armazenado no valor parcial
    # e a saída de cada unidade computacional também será armazenada para uso posterior no backpropagation
    output = []

    resultado_parcial = [[[] for j in range(hidden_layer_size[i])] for i in range(hidden_size)]

    resultado_parcial.append([[]for i in range(output_size)])

    # calcular o resultado camada por camada

    for i in range(hidden_size+1):

      # no primeiro caso será a camada de dados

      if i == 0:

        input = data[j]

      # o caso em que estamos calculando o resuldado parcial das camadas intermediárias

      if(i < hidden_size):

        for j in range(len(input)):

          for k in range(hidden_layer_size[i]):

            resultado_parcial[i][k].append(weights[i][j][k][0]*input[j])

        # aux vai receber o resultado após a função de ativação

        aux = np.zeros(hidden_layer_size[i])

        # após ter os resultados parciais da soma ponderada das entradas daquela camada, esse valor irá passar pela função de ativação

        if activation_functions[0] == 'relu':

          for j in range(hidden_layer_size[i]):

            aux[j] = function_Relu(np.sum(resultado_parcial[i][j]))

        elif activation_functions[0] == 'sigmoid':

          for j in range(hidden_layer_size[i]):

            aux[j] = function_Sigmoid(np.sum(resultado_parcial[i][j]))

        elif activation_functions[0] == 'linear':

          for j in range(hidden_layer_size[i]):

            aux[j] = function_Linear(np.sum(resultado_parcial[i][j]))


      # para o caso contrário

      else:

        for j in range(len(input)):

          for k in range(output_size):

            resultado_parcial[i][k].append(weights[i][j][k][0]*input[j])

        aux = np.zeros(output_size)

        # após ter os resultados parciais da soma ponderada das entradas daquela camada, esse valor irá passar pela função de ativação

        if activation_functions[0] == 'relu':

          for j in range(output_size):

            aux[j] = function_Relu(np.sum(resultado_parcial[i][j]))

        elif activation_functions[0] == 'sigmoid':

          for j in range(output_size):

            aux[j] = function_Sigmoid(np.sum(resultado_parcial[i][j]))

        elif activation_functions[0] == 'linear':

          for j in range(output_size):

            aux[j] = function_Linear(np.sum(resultado_parcial[i][j]))

      # após ter o resultado parcial da camada, o input da próxima iteração será o resultado dessa

      input = aux


      # output vai ter os outputs de cada camada da rede
      # o primeiro index é para a camada, apenas a primeira camada intermediária e posteriores
      # o segundo index é para a unidade computacional
      # o terceiro index é sempre 0 e refere ao valor
      output.append(input)

    resultado_final.append(output)

  return resultado_final


In [ ]:
# fase backward

def backward(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, weights, predictions, learning_rate, targets):

  new_weights = weights

  # para cada elemento de predictions fazer a correção dos pesos

  for i in range(len(predictions)):

    # a variação dos pesos vai ser igual a multiplicação do learning_rate pelas derivadas das funções e a derivada do erro em relação a saída
    # para camadas intermediárias a saída ta conectada a todos os neurónios da proxima camada, então vai ter um somatório em relação ao erro

    # vai armazenar o erro de cara unidade computacional de cada camada
    parcial_loss = [[np.zeros(1) for k in range(hidden_layer_size[j])] for j in range(hidden_size)]

    parcial_loss.append([np.zeros(1) for j in range(output_size)])

    # para cada camada da rede, ir alterando os valores dos pesos

    for j in range(hidden_size+1):

      # alterando os pesos da camada de saída

      if j == 0:

        # k refere a camada anterior a camada de saída

        for k in range(len(hidden_layer_size[-1])):

          # l refere a camada de saída

          for l in range(len(output_size)):

            # derivada do erro pelo output gerado pela camada de saída, considerando o squared_error

            derivate_error_output = predictions[i][-1][l][0]-targets[i][l]

            # derivada da função pelo input (gerado pela camada anterior) da unidade computacional

            if activation_functions[-1] == 'relu':

              derivate_function_output = derivative_Relu(predictions[i][-1][l][0])

            elif activation_functions[-1] == 'sigmoid':

              derivate_function_output = derivative_Sigmoid(predictions[i][-1][l][0])

            elif activation_functions[-1] == 'linear':

              derivate_function_output = derivative_Linear(predictions[i][-1][l][0])

            # derivada do input (gerado pela camada anterior) pelo peso analisado em questão

            derivate_input_weight = predictions[i][-2][k][0]*new_weights[-1][k][l][0]

            # atualizando o peso

            new_weights[-1][k][l][0] = new_weights[-1][k][l][0] - learning_rate*derivate_error_output*derivate_function_output*derivate_input_weight

            #armazenar o valor para calcular a derivada da camada anterior

            parcial_loss[-1][k] = derivate_error_output*derivate_function_output

      # caso a camada seja uma camada intermediária

      else:

        # k refere a camada anterior a camada queremos mudar os pesos

        for k in range(len(hidden_layer_size[-2 - j])):

          # l refere a camada de saída

          for l in range(len(hidden_layer_size[-1 - j])):

            # derivada do erro pelo output gerado pela camada de saída, considerando o squared_error
            # como para camadas intermediárias, o erro gerado por elas afeta todas as unidades computacionais da próxima camada
            # vai ocorrer um somatório desse erro

            derivate_error_output = 0

            for m in range(len(hidden_layer_size[-1 - j])):

              derivate_error_output += predictions[i][k][m][0]*weights[-1 -j][k][m][0]


            # a partir daqui eu não alterei ainda, precisa de alterações

            # derivada da função pelo input (gerado pela camada anterior) da unidade computacional

            if activation_functions[-1] == 'relu':

              derivate_function_output = derivative_Relu(predictions[i][-1][l][0])

            elif activation_functions[-1] == 'sigmoid':

              derivate_function_output = derivative_Sigmoid(predictions[i][-1][l][0])

            elif activation_functions[-1] == 'linear':

              derivate_function_output = derivative_Linear(predictions[i][-1][l][0])

            # derivada do input (gerado pela camada anterior) pelo peso analisado em questão

            derivate_input_weight = predictions[i][-2][k][0]*new_weights[-1][k][l][0]

            # atualizando o peso

            new_weights[-1][k][l][0] = new_weights[-1][k][l][0] - learning_rate*derivate_error_output*derivate_function_output*derivate_input_weight



  return new_weights

In [8]:
# o treinamento fará no máximo MAX_EPOCH iterações
# ele irá parar quando a acurácia dos testes diminuir em 5 épocas seguidas
# após o processo de treinamento será retornado os pesos ótimos para o problema encontrado

def treinamento(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, MAX_EPOCH, learning_rate, batch_size_train, batch_size_test, weights, data_train, data_test, data_train_label, data_test_label):

  for epoch in range(MAX_EPOCH):

    # atualizando os mini-batchs de treino e de teste

    batch_index = rng.choice(len(data_train), size = batch_size_train, replace = False)

    batch_train = data_train[batch_index]

    batch_train_label = data_train_label[batch_index]

    batch_index = rng.choice(len(data_test), size = batch_size_test, replace = False)

    batch_test = data_test[batch_index]

    batch_test_label = data_test_label[batch_index]

    #resultado da fase foward

    predictions = foward(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, weights, batch_train)

    # calculando o erro gerado

    error = mean_square_loss(batch_train_label, predictions)

    # atualizando os pesos, fase backward


  return weights

In [ ]:
treinamento(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, MAX_EPOCH, learning_rate, data, data, weights, data, data, data, data)

[0.71071042]


[array([[[0.70233802]],
 
        [[0.60370907]]]),
 array([[[0.54416906]]])]